In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import boto3
import datetime
import pandas as pd

In [0]:
# Retrieve the parameter values
job_id = dbutils.widgets.get("job_id")
usecase_id = dbutils.widgets.get("usecase_id")

# Print the parameter values
print(f"job_id: {job_id}")
print(f"usecase_id: {usecase_id}")

In [0]:
#convert sucess sql qyery to dataframe
success_query = f"select external_id::string as external_id,account_priority_score_vod__c::int as account_priority_score_vod__c,account_vod__c::string	as 	account_vod__c,actioned_vod__c::string	as 	actioned_vod__c,action_count_vod__c::string	as 	action_count_vod__c,call_objective_clm_id_vod__c::string	as 	call_objective_clm_id_vod__c,date_format(call_objective_from_date_vod__c,'MM/dd/yyyy') as call_objective_from_date_vod__c,call_obj_on_by_default_vod__c::string	as 	call_obj_on_by_default_vod__c,call_obj_record_type_vod__c::string	as 	call_obj_record_type_vod__c,date_format(call_objective_to_date_vod__c,'MM/dd/yyyy') as call_objective_to_date_vod__c,dismissed_vod__c::string	as 	dismissed_vod__c,dismiss_count_vod__c::string	as 	dismiss_count_vod__c,display_dismiss_vod__c::string	as 	display_dismiss_vod__c,display_mark_as_complete_vod__c::string	as 	display_mark_as_complete_vod__c,display_score_vod__c::string	as 	display_score_vod__c,email_template_id_vod__c::string	as 	email_template_id_vod__c,email_template_vault_id_vod__c::string	as 	email_template_vault_id_vod__c,email_template_vod__c::string	as 	email_template_vod__c,date_format(expiration_date_vod__c,'MM/dd/yyyy') as expiration_date_vod__c ,marked_as_complete_vod__c::string	as 	marked_as_complete_vod__c,mark_complete_count_vod__c::string	as 	mark_complete_count_vod__c,no_homepage_vod__c::string	as 	no_homepage_vod__c,date_format(planned_call_date_vod__c,'MM/dd/yyyy') as planned_call_date_vod__c ,date_format(posted_date_vod__c,'MM/dd/yyyy') as posted_date_vod__c,priority_vod__c::string	as 	priority_vod__c,reason_vod__c::string	as 	reason_vod__c,recordtypeid_suggestion::string	as 	recordtypeid_suggestion,record_type_name_vod__c_suggest::string	as 	record_type_name_vod__c_suggest,suggest_external_id_vod__c::string	as 	suggest_external_id_vod__c,suppress_reason_vod__c::string	as 	suppress_reason_vod__c,title_vod__c::string	as 	title_vod__c,ownerid::string	as 	ownerid,detail_group_vod__c::string	as 	detail_group_vod__c,driver_vod__c::string	as 	driver_vod__c,email_fragment_order_vod__c::string	as 	email_fragment_order_vod__c,email_fragment_vod__c::string	as 	email_fragment_vod__c,suggest_tag_external_id_vod__c::string	as 	suggest_tag_external_id_vod__c,product_vod__c::string	as 	product_vod__c,record_type_name_vod__c_sug_tag::string	as 	record_type_name_vod__c_sug_tag,recordtypeid_suggestion_tag::string	as 	recordtypeid_suggestion_tag,analytics_title::string	as 	analytics_title,suggestion_survey_vod__c::string	as 	suggestion_survey_vod__c,Lead_Id::string	as 	Lead_Id from com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed where Sent_To_Veeva = current_date and Not_Sent_To_Veeva_Reason is NULL and usecase_id = '{usecase_id}';"
success_query_df = sqlContext.sql(success_query).collect()
# Define the column headers
headers_sucess = ['external_id', 'account_priority_score_vod__c', 'account_vod__c','actioned_vod__c','action_count_vod__c','call_objective_clm_id_vod__c','call_objective_from_date_vod__c','call_obj_on_by_default_vod__c','call_obj_record_type_vod__c','call_objective_to_date_vod__c','dismissed_vod__c','dismiss_count_vod__c','display_dismiss_vod__c','display_mark_as_complete_vod__c','display_score_vod__c','email_template_id_vod__c','email_template_vault_id_vod__c','email_template_vod__c','expiration_date_vod__c','marked_as_complete_vod__c','mark_complete_count_vod__c','no_homepage_vod__c','planned_call_date_vod__c','posted_date_vod__c','priority_vod__c','reason_vod__c','recordtypeid_suggestion','record_type_name_vod__c_suggest','suggest_external_id_vod__c','suppress_reason_vod__c','title_vod__c','ownerid','detail_group_vod__c','driver_vod__c','email_fragment_order_vod__c','email_fragment_vod__c','suggest_tag_external_id_vod__c','product_vod__c','record_type_name_vod__c_sug_tag','recordtypeid_suggestion_tag','analytics_title','suggestion_survey_vod__c','Lead_Id']  

if len(success_query_df) > 0:
    # Create a list to store the data
    success_list = []
    for curr_row in success_query_df:
        success_list.append(curr_row.asDict(True))

    # Create the DataFrame with data and headers
    success_df = pd.DataFrame(success_list)
else:
    # Create an empty DataFrame with headers
    success_df = pd.DataFrame(columns=headers_sucess)

#convert Error sql qyery to dataframe
error_query = f"WITH t(usecase_id, hist_retention) AS (SELECT USECASE_ID,hist_retention from com_us_alyt_ngebox.metadata_usecase) select * from com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed where usecase_id = '{usecase_id}' and DATE(created_date) >=  date_add(current_date, -(select hist_retention from t where t.usecase_id = '{usecase_id}' )) and Sent_To_Veeva is null and processed = 1 and Not_Sent_To_Veeva_Reason is not null "
error_query_df = sqlContext.sql(error_query).collect()

headers_error = ['NGEBox_Record_ID','usecase_id','HCP_NPI','HCP_OMNI_ID','HCP_ZIP_CODE','external_id','account_priority_score_vod__c','account_vod__c','actioned_vod__c',
'action_count_vod__c','call_objective_clm_id_vod__c','call_objective_from_date_vod__c','call_obj_on_by_default_vod__c','call_obj_record_type_vod__c','call_objective_to_date_vod__c','dismissed_vod__c','dismiss_count_vod__c','display_dismiss_vod__c','display_mark_as_complete_vod__c','display_score_vod__c','email_template_id_vod__c','email_template_vault_id_vod__c','email_template_vod__c''expiration_date_vod__c','marked_as_complete_vod__c','mark_complete_count_vod__c','no_homepage_vod__c','planned_call_date_vod__c','posted_date_vod__c','priority_vod__c','reason_vod__c','recordtypeid_suggestion','record_type_name_vod__c_suggest','suggest_external_id_vod__c','suppress_reason_vod__c','title_vod__c','ownerid','detail_group_vod__c','driver_vod__c','email_fragment_order_vod__c','email_fragment_vod__c','suggest_tag_external_id_vod__c','product_vod__c','record_type_name_vod__c_sug_tag','recordtypeid_suggestion_tag','analytics_title','suggestion_survey_vod__c','Lead_Id','Processed',
'Not_Sent_To_Veeva_Reason','Sent_to_Veeva','Created_Date','Created_By','HCP_NPI_OLD','HCP_OMNI_ID_OLD','OWNERID_OLD','HR_Hold_Flag','PDRP_Restriction_Flag','TKD_Sanction_Flag']
if len(error_query_df) > 0:
    # Create a list to store the data
    error_list = []
    for curr_row in error_query_df:
        error_list.append(curr_row.asDict(True))

    # Create the DataFrame with data and headers
    error_df = pd.DataFrame(error_list)
else:
    # Create an empty DataFrame with headers
    error_df = pd.DataFrame(columns=headers_error)

In [0]:
# get s3 params from metadata table 
s3_query = f"SELECT success_filename_prefix,error_filename_prefix,s3_bucket_name,s3_STG_path,s3_archive_path from com_us_alyt_ngebox.metadata_usecase where USECASE_ID = '{usecase_id}'"
s3_bucket_query_df = sqlContext.sql(s3_query).collect()[0]['s3_bucket_name']
s3_STG_path_query_df = sqlContext.sql(s3_query).collect()[0]['s3_STG_path']
s3_archive_path_query_df = sqlContext.sql(s3_query).collect()[0]['s3_archive_path']
s3_success_prefix_df = sqlContext.sql(s3_query).collect()[0]['success_filename_prefix']
s3_error_prefix_df = sqlContext.sql(s3_query).collect()[0]['error_filename_prefix']

# Set the S3 bucket and folder path
s3_bucket = s3_bucket_query_df
s3_folder_path_STG = s3_STG_path_query_df
s3_folder_path_arc = s3_archive_path_query_df
s3_success_filename = s3_success_prefix_df
s3_error_filename = s3_error_prefix_df

# Get the current date and time
folder_name = datetime.datetime.now().strftime("%Y-%m-%d")

# Create the S3 client
s3_client = boto3.client('s3')

# Create the CSV file name using the current date
now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
success_csv_file_name = f"{s3_success_filename}{now}.csv"	
error_csv_file_name = f"{s3_error_filename}{now}.csv"

# Convert the DataFrame to CSV data
success_csv_data = success_df.to_csv(index=False)
error_csv_data = error_df.to_csv(index=False)

# Write the dynamic CSV file to the S3 bucket folder for success and error files
s3_client.put_object(Bucket=s3_bucket, Key=s3_folder_path_arc + folder_name + '/' + success_csv_file_name, Body=success_csv_data)
s3_client.put_object(Bucket=s3_bucket, Key=s3_folder_path_arc + folder_name + '/' + error_csv_file_name, Body=error_csv_data)

# Remove the existing CSV files in the STG path
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=s3_folder_path_STG)
for obj in response.get('Contents', []):
    if obj['Key'].endswith('.csv'):
        s3_client.delete_object(Bucket=s3_bucket, Key=obj['Key'])
        print(f"Deleted object: {obj['Key']}")

# Copy the new CSV file to the STG path
s3_client.put_object(Bucket=s3_bucket, Key=s3_folder_path_STG + success_csv_file_name, Body=success_csv_data)

Veeva_Suggestion_File_Takhzyro_Leads_labs_
Veeva_Suggestion_File_Takhzyro_Leads_labs_
Out[15]: {'ResponseMetadata': {'RequestId': 'WJA7R71BZR1B0J4F',
  'HostId': 's0fA62hhnvQ75ncw+7PGqmRJyKIPXsrXVThYvrfiP2xoNA48buFuGsVSwpJVFhyPh7ZdF8dSmYw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 's0fA62hhnvQ75ncw+7PGqmRJyKIPXsrXVThYvrfiP2xoNA48buFuGsVSwpJVFhyPh7ZdF8dSmYw=',
   'x-amz-request-id': 'WJA7R71BZR1B0J4F',
   'date': 'Thu, 22 Jun 2023 11:59:37 GMT',
   'x-amz-version-id': 'THuI6nJz3uTeRKq8_KppfjQ.Oe37ncJy',
   'x-amz-server-side-encryption': 'aws:kms',
   'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:us-east-1:263789222982:key/253920df-b84a-47ca-a26e-ace8b712a886',
   'etag': '"87b39831bd7d79c6a1021be8ea16a09f"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"87b39831bd7d79c6a1021be8ea16a09f"',
 'ServerSideEncryption': 'aws:kms',
 'VersionId': 'THuI6nJz3uTeRKq8_KppfjQ.Oe37ncJy',
 'SSEKMSKeyId': 'arn:aws:kms:us-east-1:2637

In [0]:
class s3Load:
   def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": "s3 file load completed"
      }
    }
    self.response = resp

In [0]:
metadata_obj = s3Load()
metadata_obj.construct_microservice_response()
dbutils.notebook.exit(
  metadata_obj.response
)

{'nge_response': {'status': 200, 'body': 's3 file load completed'}}